In [1]:
from mace.calculators import mace_mp
from ase import build

from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase import Atoms
from ase.build import bulk
from ase.visualize import view
import numpy as np
from ase.build import add_vacuum
from ase.optimize import LBFGS, BFGS
from ase import Atom

from ase.io.trajectory import Trajectory
from ase.md import Langevin, Bussi
from ase import io


#macemp = mace_mp(dispersion=True, default_dtype="float64")
macemp_omat = mace_mp(model="/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/mace-omat-0-medium.model", dispersion=True, default_dtype="float64")

#macemp_omat = mace_mp(model="/home/jh2536/rds/hpc-work/mace_data/models/mace-omat-0-medium.model", dispersion=True, default_dtype="float64")

/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


In [2]:
mxene_relaxed = io.read("structures/relaxed_mxene.xyz")
graphene_atoms = io.read("structures/graphene_atoms.xyz")
go_relax = io.read("structures/go_omat_relax.xyz")
goh_relax = io.read("structures/goh_omat_relax.xyz")

In [ ]:
def create_heterostructure(mxene, graphene_layer, height):
    # Stack MXene and graphene-based layer with a given interlayer spacing (height)
    graphene_layer = graphene_layer.copy()
    
    mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.tag == 3])

    #mxene_avg_top_C_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'C'])
    #mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'O' and atom.position[2] > mxene_avg_top_C_z])
    
    graphene_layer.positions[:, 2] += mxene_avg_top_O_z - np.average(graphene_layer.positions[:, 2]) + height
    heterostructure = mxene + graphene_layer
    print("MXene avg top O z: ", mxene_avg_top_O_z)

    return heterostructure


m_g = create_heterostructure(mxene_relaxed, graphene_atoms, 4.5)
m_goh = create_heterostructure(mxene_relaxed, goh_relax, 4.5)


#--------------------------------Geometry optimisation--------------------------------

m_g_relaxed = m_g.copy()
m_g_relaxed.calc = macemp_omat
optimiser = BFGS(m_g_relaxed)
optimiser.run(fmax=0.001, steps=5000)

MXene avg top O z:  12.23008024


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/torch_dftd/torch_dftd3_calculator.py:98: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  cell: Optional[Tensor] = torch.tensor(


      Step     Time          Energy          fmax
BFGS:    0 19:45:15    -1195.136431       10.049981
BFGS:    1 19:45:17    -1196.948171        8.295128
BFGS:    2 19:45:19    -1201.058147        2.613683
BFGS:    3 19:45:21    -1201.426459        1.849330
BFGS:    4 19:45:23    -1201.644086        0.567425
BFGS:    5 19:45:25    -1201.725101        0.374767
BFGS:    6 19:45:27    -1201.752619        0.233648
BFGS:    7 19:45:29    -1201.763044        0.162014
BFGS:    8 19:45:31    -1201.777831        0.196212
BFGS:    9 19:45:33    -1201.791782        0.178716
BFGS:   10 19:45:35    -1201.797459        0.111643
BFGS:   11 19:45:37    -1201.801383        0.092853
BFGS:   12 19:45:39    -1201.807753        0.180901
BFGS:   13 19:45:41    -1201.821566        0.282898
BFGS:   14 19:45:43    -1201.847199        0.414836
BFGS:   15 19:45:45    -1201.891389        0.504348
BFGS:   16 19:45:48    -1201.962133        0.528660
BFGS:   17 19:45:50    -1202.066505        0.523069
BFGS:   18 19:

True

In [26]:
m_goh = create_heterostructure(mxene_relaxed, goh_relax, 4.5)


m_goh_relaxed = m_goh.copy()
m_goh_relaxed.calc = macemp_omat
optimiser = BFGS(m_goh_relaxed)
optimiser.run(fmax=0.001, steps=5000)

MXene avg top O z:  12.23008024
      Step     Time          Energy          fmax
BFGS:    0 18:14:04    -1202.749183       10.125747
BFGS:    1 18:14:06    -1204.843316        8.135644
BFGS:    2 18:14:09    -1208.570336        2.538501
BFGS:    3 18:14:11    -1208.954129        1.788395
BFGS:    4 18:14:13    -1209.175802        0.979869
BFGS:    5 18:14:15    -1209.297966        0.970648
BFGS:    6 18:14:17    -1209.350343        0.878384
BFGS:    7 18:14:19    -1209.403783        0.742956
BFGS:    8 18:14:21    -1209.477782        0.546270
BFGS:    9 18:14:23    -1209.535286        0.487209
BFGS:   10 18:14:26    -1209.582862        0.297061
BFGS:   11 18:14:28    -1209.596058        0.279873
BFGS:   12 18:14:30    -1209.632031        0.417715
BFGS:   13 18:14:33    -1209.653944        0.443394
BFGS:   14 18:14:35    -1209.674853        0.486805
BFGS:   15 18:14:37    -1209.698156        0.428347
BFGS:   16 18:14:39    -1209.730835        0.668541
BFGS:   17 18:14:41    -1209.76981

True

In [9]:
view(m_g_relaxed, viewer='x3d')

In [3]:
from ase.constraints import FixSubsetCom

In [4]:
m_g_relaxed.get_tags()

NameError: name 'm_g_relaxed' is not defined

In [24]:
mxene_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])

In [25]:
# def printenergy(a=m_g_relaxed):
#     """Function to print the potential, kinetic and total energy"""
#     epot = a.get_potential_energy() / len(a)
#     ekin = a.get_kinetic_energy() / len(a)
#     print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
#           'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

import time
start_time = time.time()

def printenergy(a=m_g_relaxed):
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    elapsed_time = time.time() - start_time  # Time in seconds
    elapsed_min, elapsed_sec = divmod(elapsed_time, 60)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV Time Elapsed: %dm %.1fs' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin, int(elapsed_min), int(elapsed_sec)))



# m_g
#m_g_relaxed.calc = macemp
m_g_relaxed.calc = macemp_omat
T_init = 300  # Initial temperature in K
MaxwellBoltzmannDistribution(m_g_relaxed, temperature_K=T_init)
#remove_COM_momentum_xy(m_g_relaxed)
#Stationary(m_g_relaxed)

tags = m_g_relaxed.get_tags()

mxene_indices = np.where(np.isin(tags, [0, 1, 2, 3]))[0]  # MXene atoms
graphene_indices = np.where(np.isin(tags, [10, 11, 12]))[0]  # Graphene atoms
#m_g_relaxed.set_constraint(FixSubsetCom(graphene_indices))

fix_com_mxene = FixSubsetCom(indices=mxene_indices)
fix_com_graphene = FixSubsetCom(indices=graphene_indices)
m_g_relaxed.set_constraint([fix_com_mxene, fix_com_graphene])

#dyn = Langevin(m_g_relaxed, 1 * units.fs, T_init * units.kB, 0.01)
dyn = Bussi(m_g_relaxed, 0.5 * units.fs, T_init, 0.01)
#n_steps = 5000 + 20000 # 5ps pre-equilibration + 20ps production
n_steps = 1000

dyn.attach(printenergy, interval = 100)
#dyn.attach(lambda: remove_COM_momentum(m_g_relaxed), interval=10)

traj = Trajectory('mg_Bussi1000.traj', 'w', m_g_relaxed)
dyn.attach(traj.write, interval = 10)
printenergy()
dyn.run(n_steps)

Energy per atom: Epot = -9.213eV  Ekin = 0.037eV (T=283K)  Etot = -9.177eV Time Elapsed: 0m 0.0s
Energy per atom: Epot = -9.213eV  Ekin = 0.037eV (T=283K)  Etot = -9.177eV Time Elapsed: 0m 0.0s
Energy per atom: Epot = -9.214eV  Ekin = 0.034eV (T=265K)  Etot = -9.179eV Time Elapsed: 3m 6.0s
Energy per atom: Epot = -9.207eV  Ekin = 0.034eV (T=265K)  Etot = -9.173eV Time Elapsed: 5m 57.0s
Energy per atom: Epot = -9.211eV  Ekin = 0.039eV (T=304K)  Etot = -9.172eV Time Elapsed: 8m 46.0s
Energy per atom: Epot = -9.216eV  Ekin = 0.039eV (T=303K)  Etot = -9.177eV Time Elapsed: 11m 35.0s
Energy per atom: Epot = -9.210eV  Ekin = 0.035eV (T=273K)  Etot = -9.174eV Time Elapsed: 14m 39.0s
Energy per atom: Epot = -9.215eV  Ekin = 0.038eV (T=296K)  Etot = -9.177eV Time Elapsed: 17m 36.0s
Energy per atom: Epot = -9.213eV  Ekin = 0.038eV (T=291K)  Etot = -9.175eV Time Elapsed: 20m 33.0s
Energy per atom: Epot = -9.211eV  Ekin = 0.033eV (T=257K)  Etot = -9.178eV Time Elapsed: 23m 36.0s
Energy per atom: E

True

In [4]:
from ase.constraints import FixConstraint
import numpy as np

class FixXYCom(FixConstraint):
    def __init__(self, indices):
        """
        Constraint that fixes the center of mass (COM) in the x and y directions
        for a subset of atoms while allowing movement in the z-direction.

        Parameters:
        - indices: List or array of atom indices whose COM will be constrained
        """
        self.indices = np.array(indices)
        self.initial_com_xy = None  # Will be set during `initialize`

    def initialize(self, atoms):
        """
        Store reference to atoms and compute the initial COM in x, y.
        This is called once before MD starts.
        """
        self.atoms = atoms
        positions = atoms.get_positions()
        masses = atoms.get_masses()[self.indices]

        # Compute and store initial x, y COM
        com = np.sum(positions[self.indices] * masses[:, None], axis=0) / np.sum(masses)
        self.initial_com_xy = com[:2]

    def adjust_positions(self, atoms, newpositions):
        """
        Ensure that the x, y COM remains fixed.
        This function is called during each MD step.
        
        Parameters:
        - atoms: ASE Atoms object
        - newpositions: Updated positions array from ASE optimizer
        """
        positions = newpositions[self.indices]
        masses = self.atoms.get_masses()[self.indices]

        # Compute new COM in x, y
        com = np.sum(positions * masses[:, None], axis=0) / np.sum(masses)

        # Compute displacement to correct x, y COM
        displacement_xy = self.initial_com_xy - com[:2]

        # Apply correction only to x, y coordinates
        newpositions[self.indices, :2] += displacement_xy  

    def get_removed_dof(self, atoms):
        """
        Return the number of removed degrees of freedom.
        
        Since we fix only the x and y components of the COM, we remove exactly 2 DOF.
        """
        return 2

    def todict(self):
        """
        Convert the constraint to a dictionary for serialization.
        ASE requires this for MD compatibility.
        """
        return {
            '@module': self.__module__,
            '@class': self.__class__.__name__,
            'indices': self.indices.tolist()
        }


In [7]:
from ase.constraints import FixConstraint
import numpy as np

class FixXYCom(FixConstraint):
    def __init__(self, indices):
        """
        Constraint that fixes the center of mass (COM) in the x and y directions
        for a subset of atoms while allowing movement in the z-direction.

        Parameters:
        - indices: List or array of atom indices whose COM will be constrained
        """
        self.indices = np.array(indices)
        self.initial_com_xy = None
        self.masses = None  # Store masses to avoid repeated lookups

    def initialize(self, atoms):
        """
        Store reference to atoms and precompute masses for efficiency.
        """
        self.atoms = atoms
        self.masses = atoms.get_masses()[self.indices]  # Store masses
        positions = atoms.get_positions()

        # Compute and store initial x, y COM
        self.initial_com_xy = np.dot(self.masses, positions[self.indices, :2]) / np.sum(self.masses)

    def adjust_positions(self, atoms, newpositions):
        """
        Ensure that the x, y COM remains fixed.
        This function is called during each MD step.
        
        Parameters:
        - atoms: ASE Atoms object
        - newpositions: Updated positions array from ASE optimizer
        """
        positions = newpositions[self.indices, :2]  # Only x, y
        com_xy = np.dot(self.masses, positions) / np.sum(self.masses)  # Fast COM calculation

        # Compute displacement needed to fix x, y COM
        displacement_xy = self.initial_com_xy - com_xy

        # Apply correction only to x, y coordinates
        newpositions[self.indices, :2] += displacement_xy  

    def get_removed_dof(self, atoms):
        """Return the number of removed degrees of freedom (2)."""
        return 2

    def todict(self):
        """Convert the constraint to a dictionary for serialization."""
        return {
            '@module': self.__module__,
            '@class': self.__class__.__name__,
            'indices': self.indices.tolist()
        }


In [3]:
m_g_relaxed = io.read("structures/m_g_relaxed_omat.xyz")

In [ ]:
import time
start_time = time.time()

def printenergy(a=m_g_relaxed):
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    elapsed_time = time.time() - start_time  # Time in seconds
    elapsed_min, elapsed_sec = divmod(elapsed_time, 60)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV Time Elapsed: %dm %.1fs' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin, int(elapsed_min), int(elapsed_sec)))



m_g_relaxed.calc = macemp_omat
T_init = 300
MaxwellBoltzmannDistribution(m_g_relaxed, temperature_K=T_init)


tags = m_g_relaxed.get_tags()

mxene_indices = np.where(np.isin(tags, [0, 1, 2, 3]))[0]  
graphene_indices = np.where(np.isin(tags, [10, 11, 12]))[0] 

fix_xy_com_mxene = FixXYCom(mxene_indices)
fix_xy_com_graphene = FixXYCom(graphene_indices)
m_g_relaxed.set_constraint([fix_xy_com_mxene, fix_xy_com_graphene])
fix_xy_com_mxene.initialize(m_g_relaxed)
fix_xy_com_graphene.initialize(m_g_relaxed)

dyn = Bussi(m_g_relaxed, 1 * units.fs, T_init, 0.01)
n_steps = 10

dyn.attach(printenergy, interval = 1)

traj = Trajectory('mg_contrain_xy_z.traj', 'w', m_g_relaxed)
dyn.attach(traj.write, interval = 1)
printenergy()
dyn.run(n_steps)

Energy per atom: Epot = -9.249eV  Ekin = 0.042eV (T=322K)  Etot = -9.207eV Time Elapsed: 0m 0.0s
Energy per atom: Epot = -9.249eV  Ekin = 0.042eV (T=322K)  Etot = -9.207eV Time Elapsed: 0m 0.0s


KeyboardInterrupt: 